# shop
# 데이터세트 합치기

In [23]:
cd /Users/limkahyun/Desktop/small

/Users/limkahyun/Desktop/small


In [31]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

%matplotlib inline
%config Completer.use_jedi = False

In [26]:
# 여러 csv파일 하나로 합치기

# 파일 모으기
path = 'shop'
file_all = glob.glob(path + '**/**.csv')

#file_all = glob.glob('*csv')

print(file_all)

['shop/2015년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2014년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2018년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2016년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2019년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2017년_우리마을가게_상권서비스_상권배후지-추정매출.csv', 'shop/2020년_우리마을가게_상권서비스_상권배후지-추정매출.csv']


In [27]:
# 빈 데이터프레임을 만들어서 넣어주기

dataframe = []

for file in file_all:
    df = pd.read_csv(file, encoding='cp949')
    dataframe.append(df)
    
data_all = pd.concat(dataframe, axis=0)
    
# 다 합친 파일을 저장할 위치 및 이름
output_file = 'shop_all.csv'

# 모두 합친 데이터를 csv로 저장하기
data_all.to_csv(output_file, index=False)

# 데이터 가공

In [4]:
# 데이터 불러오기
shop_orig = pd.read_csv('shop_all.csv')
shop_all = shop_orig.copy()

### 점포수 0인것 1로 변환

In [29]:
shop_all['점포수'].value_counts().sort_index()

0        8657
1       55858
2       76972
3       88642
4       93505
        ...  
2520        1
2546        2
2550        1
2592        1
2618        1
Name: 점포수, Length: 893, dtype: int64

In [30]:
# 점포수 0인것 1로 변경
shop_all['점포수'].replace(0, 1, inplace=True)
shop_all['점포수'].value_counts().sort_index()

1       64515
2       76972
3       88642
4       93505
5       91103
        ...  
2520        1
2546        2
2550        1
2592        1
2618        1
Name: 점포수, Length: 892, dtype: int64

### 컬럼 정리

In [32]:
shop_col = shop_all.copy()

# 필요한 컬럼 생성
shop_col['period'] = shop_col['기준_년_코드'].astype(str) + '-' + shop_col['기준_분기_코드'].astype(str)
shop_col['shop'] = shop_col['상권_코드'].astype(str) + '-' + shop_col['서비스_업종_코드_명'].astype(str)
shop_col['mean_money'] = shop_col['당월_매출_금액'] / shop_col['점포수']

# 필요없는 컬럼 삭제
shop_col.drop(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
           '서비스_업종_코드', '서비스_업종_코드_명'], axis=1, inplace=True)

In [33]:
# 상관관계 확인 >> 비율은 관련이 적고 건수는 관련이 높음
corr = shop_col.corr()
print(corr.mean_money.sort_values(ascending=False)[:30])
print()
print(corr.mean_money.sort_values(ascending=False)[30:73])

mean_money         1.000000
시간대_06~11_매출_금액    0.268901
연령대_50_매출_건수       0.259353
시간대_건수~17_매출_건수    0.253964
연령대_40_매출_건수       0.246748
월요일_매출_건수          0.242464
주중_매출_금액           0.238145
금요일_매출_건수          0.237994
토요일_매출_건수          0.237534
당월_매출_건수           0.237418
주중_매출_건수           0.237238
당월_매출_금액           0.235297
화요일_매출_건수          0.235060
목요일_매출_건수          0.233854
수요일_매출_건수          0.233813
목요일_매출_금액          0.233738
연령대_60_이상_매출_건수    0.233410
연령대_50_매출_금액       0.232589
금요일_매출_금액          0.231684
화요일_매출_금액          0.231614
월요일_매출_금액          0.231401
여성_매출_금액           0.231336
수요일_매출_금액          0.228703
여성_매출_건수           0.228550
시간대_건수~11_매출_건수    0.226915
시간대_건수~21_매출_건수    0.224932
주말_매출_건수           0.224619
남성_매출_건수           0.223159
시간대_11~14_매출_금액    0.217179
연령대_60_이상_매출_금액    0.213298
Name: mean_money, dtype: float64

남성_매출_금액           0.212898
연령대_30_매출_건수       0.212243
시간대_건수~14_매출_건수    0.208210
연령대_30_매출_금액       0.206570
일요일_매출_건수     

In [ ]:
shop_col.columns

In [34]:
# 매출 금액만 남기고 모두 삭제
shop_drop = shop_col.copy()
shop_drop.drop(['당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '월요일_매출_비율','화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
                '금요일_매출_비율', '토요일_매출_비율','일요일_매출_비율', '시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율',
                '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율', '연령대_10_매출_비율',
                '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율', '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_건수', 
                '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', 
                '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수', 
                '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', 
                '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '점포수', '월요일_매출_금액', '화요일_매출_금액', 
                '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', 
                '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', 
                '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'],
               axis=1, inplace=True)

shop_drop.head()

,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,period,shop,mean_money
0,284484215,198729159,85755056,106969928,169353641,2015-1,1000428-제과점,4.064060e+07
1,12005909,9736459,2269450,4839538,6868387,2015-1,1000288-섬유제품,3.001477e+06
2,71547516,50879960,20667556,44823579,17296999,2015-1,1000270-자동차수리,1.022107e+07
3,80195548,58480589,21714959,51194195,19312565,2015-1,1000284-자동차수리,1.002444e+07
4,1339105488,909912741,429192747,881901743,420784068,2015-1,1000287-편의점,1.217369e+08


In [ ]:
X.shape

In [35]:
# 분기를 인덱스 오름차순으로 설정
shop_drop.index = shop_drop['period']
shop_drop = shop_drop.sort_index(ascending=True)

In [36]:
shop_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1434810 entries, 2014-1 to 2020-2
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   당월_매출_금액    1434810 non-null  int64  
 1   주중_매출_금액    1434810 non-null  int64  
 2   주말_매출_금액    1434810 non-null  int64  
 3   남성_매출_금액    1434810 non-null  int64  
 4   여성_매출_금액    1434810 non-null  int64  
 5   period      1434810 non-null  object 
 6   shop        1434810 non-null  object 
 7   mean_money  1434810 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 98.5+ MB


### 레이블인코딩

In [38]:
from sklearn.preprocessing import LabelEncoder

# 레이블인코딩
enco = LabelEncoder()
shop_drop['period'] = enco.fit_transform(shop_drop['period'])
shop_drop['shop'] = enco.fit_transform(shop_drop['shop'])

### 함수생성

In [29]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    return rmse
    
def get_mae(model):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test,pred)
    return mae

def get_mape(model):
    pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, pred)
    return mape

def get_r2(model):
    pred = model.predict(X_test)
    r2 = r2_score(y_test, pred)
    return r2

def check_errors(models):
    rmses = []
    maes = []
    mapes = []
    r2s = []
    
    for model in models:
        rmse = get_rmse(model)
        mae = get_mae(model)
        mape = get_mape(model)
        r2 = get_r2(model)
        
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        r2s.append(r2)
        
        print(model.__class__.__name__, 'RMSE :', np.round(rmse, 3), '/ MAE :', np.round(mae, 3), '/ MAPE :', np.round(mape, 3), '/ R2 :', np.round(r2, 3))
    return rmses, maes, mapes, r2

### 모델 학습, 예측, 평가

In [28]:
# 데이터 분리
y_target = shop_drop['mean_money']
X_features = shop_drop.drop(['mean_money'],axis=1,inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=0)

NameError: name 'shop_drop' is not defined

In [41]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors(models)

LinearRegression RMSE : 114672705.171 / MAE : 46505994.151 / MAPE : 130.277 / R2 : 0.065
Ridge RMSE : 114672708.019 / MAE : 46506018.209 / MAPE : 130.277 / R2 : 0.065
Lasso RMSE : 114672706.373 / MAE : 46506053.325 / MAPE : 130.278 / R2 : 0.065
XGBRegressor RMSE : 81723815.041 / MAE : 30564394.435 / MAPE : 5.723 / R2 : 0.525


([114672705.17064795,
  114672708.01894775,
  114672706.37333925,
  81723815.04114692],
 [46505994.15113868, 46506018.20856451, 46506053.32475212, 30564394.434903163],
 [130.27715547055084,
  130.27732406861185,
  130.27763036533563,
  5.722658894162153],
 0.5251132114806714)

## 레이블이 많아서 시간이 오래걸리고 정확도 매우 낮음 다른 방법으로 데이터 전처리 필요

### 다른 방법으로 데이터 전처리 시도

In [42]:
shop_col2 = shop_all.copy()

# 필요한 컬럼 생성
shop_col2['period'] = shop_col2['기준_년_코드'].astype(str) + '-' + shop_col2['기준_분기_코드'].astype(str)
shop_col2['mean_money'] = shop_col2['당월_매출_금액'] / shop_col2['점포수']

In [43]:
# 매출 금액만 남기고 모두 삭제
shop_drop2 = shop_col2.copy()
shop_drop2.drop(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드', '점포수',
                '당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '월요일_매출_비율','화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
                '금요일_매출_비율', '토요일_매출_비율','일요일_매출_비율', '시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율',
                '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율', '연령대_10_매출_비율',
                '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율', '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_건수', 
                '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', 
                '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수', 
                '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', 
                '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '월요일_매출_금액', '화요일_매출_금액', 
                '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', 
                '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', 
                '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'],
               axis=1, inplace=True)

shop_drop2.head()

,상권_코드,서비스_업종_코드_명,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,period,mean_money
0,1000428,제과점,284484215,198729159,85755056,106969928,169353641,2015-1,4.064060e+07
1,1000288,섬유제품,12005909,9736459,2269450,4839538,6868387,2015-1,3.001477e+06
2,1000270,자동차수리,71547516,50879960,20667556,44823579,17296999,2015-1,1.022107e+07
3,1000284,자동차수리,80195548,58480589,21714959,51194195,19312565,2015-1,1.002444e+07
4,1000287,편의점,1339105488,909912741,429192747,881901743,420784068,2015-1,1.217369e+08


In [44]:
# 분기를 인덱스 오름차순으로 설정
shop_drop2.index = shop_drop2['period']
shop_drop2 = shop_drop2.sort_index(ascending=True)
shop_drop2.drop(['period'], axis=1, inplace=True)
shop_drop2.head()

,상권_코드,서비스_업종_코드_명,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,mean_money
period,,,,,,,,
2014-1,1000560,중식음식점,12363015,7190594,5172421,5191066,4598513,6.181508e+06
2014-1,1000725,제과점,316665247,234156208,82509039,118884530,168038070,3.958316e+07
2014-1,1000063,화초,52043466,36531072,15512394,16000724,26244832,6.505433e+06
2014-1,1000691,여행사,26048736,26048736,0,14443127,1977252,1.532279e+06
2014-1,1000070,화초,60814678,45302284,15512394,27877778,23138990,8.687811e+06


피벗 이용 이후 resetindex로 dataframe으로 변경하면 타임시리즈로 하기 편하다  
이후 dropna필요  

In [45]:
shop_group = shop_drop2.groupby([shop_drop2['상권_코드'], shop_drop2['서비스_업종_코드_명']]).mean()

In [46]:
shop_group.astype(int)

당월_매출_금액    주중_매출_금액   주말_매출_금액    남성_매출_금액   여성_매출_금액  \
상권_코드   서비스_업종_코드_명                                                             
1000001 가구             56807859    40816993   15990865     8399734   46721004   
        가방             84372425    46616719   37755706    22286667   57884230   
        네일숍             6885714     6415126     470588           0    6885714   
        노래방            10558338     7763570    2794768     5734521    2583928   
        당구장             8484781     7262435    1222345     7913243     366528   
...                         ...         ...        ...         ...        ...   
1001010 한의원           491689188   392654378   99034809   171947446  308448984   
        핸드폰           294042255   234089688   59952567   101523340  105452484   
        호프-간이주점      1876517907  1261520741  614997166  1227311343  548276078   
        화장품           387241111   295385096   91856015    80635191  296009847   
        화초             50762690    35041504   15721185    23126801   23391944   

                     mean_money  
상권_코드   서비스_업종_코드_명              
1000001 가구             56807859  
        가방             23201150  
        네일숍             5150000  
        노래방             5223033  
        당구장             4950255  
...                         ...  
1001010 한의원            91302967  
        핸드폰            24727095  
        호프-간이주점        28872344  
        화장품            15704358  
        화초              5245042  

[63732 rows x 6 columns]

### 모델 학습, 예측, 평가

In [47]:
# 데이터 분리
y = shop_group['mean_money']
X = shop_group.drop(['mean_money'],axis=1,inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

y값을 2020년 2분기로 줘서 예측

In [48]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors(models)

LinearRegression RMSE : 112035704.202 / MAE : 46087498.53 / MAPE : 72.519 / R2 : 0.04
Ridge RMSE : 112035697.029 / MAE : 46088384.834 / MAPE : 72.522 / R2 : 0.04
Lasso RMSE : 112035696.361 / MAE : 46088389.534 / MAPE : 72.522 / R2 : 0.04
XGBRegressor RMSE : 113401830.924 / MAE : 31680953.217 / MAPE : 4.334 / R2 : 0.017


([112035704.20159073,
  112035697.02931471,
  112035696.36109258,
  113401830.92364556],
 [46087498.53001537, 46088384.83377135, 46088389.53409105, 31680953.21724263],
 [72.51862514995781, 72.52177520523057, 72.52179224407611, 4.334067440247355],
 0.016812153679671482)

## 정확도 높이기
**1. 스케일링 Standard**

In [49]:
from sklearn.preprocessing import StandardScaler

# 데이터에서 Y 제거
shop_X = shop_group.drop(['mean_money'],axis=1,inplace=False)

# StandardScaler 객체 생성
st_scaler = StandardScaler()

# StandardScaler로 데이터 세트 변환
st_scaler.fit(shop_X)
shop_st= st_scaler.transform(shop_X)

# 데이터프레임으로 변환
shop_df_st = pd.DataFrame(data=shop_st, columns=shop_X.columns)
shop_df_st.describe()

,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액
count,6.373200e+04,6.373200e+04,6.373200e+04,6.373200e+04,6.373200e+04
mean,-6.831455e-17,5.223442e-18,-8.678600e-16,-1.645654e-16,3.182319e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-2.342458e-01,-2.202893e-01,-2.638781e-01,-2.641479e-01,-3.314543e-01
25%,-2.184485e-01,-2.053161e-01,-2.408322e-01,-2.478995e-01,-3.117976e-01
50%,-1.731969e-01,-1.621742e-01,-2.008023e-01,-1.973587e-01,-2.534733e-01
75%,-2.622065e-02,-2.405781e-02,-4.015126e-02,-3.400593e-02,-3.699669e-02
max,1.046230e+02,1.029989e+02,9.624635e+01,1.167421e+02,5.943568e+01


In [50]:
# 데이터 분리
y = shop_group['mean_money']
X = shop_df_st

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [51]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors(models)

LinearRegression RMSE : 112035203.474 / MAE : 46088061.393 / MAPE : 72.521 / R2 : 0.04
Ridge RMSE : 112035056.61 / MAE : 46088926.545 / MAPE : 72.524 / R2 : 0.04
Lasso RMSE : 112035695.594 / MAE : 46088390.16 / MAPE : 72.522 / R2 : 0.04
XGBRegressor RMSE : 113401828.085 / MAE : 31680930.955 / MAPE : 4.334 / R2 : 0.017


([112035203.47417831,
  112035056.6102494,
  112035695.59365867,
  113401828.08464013],
 [46088061.39272474, 46088926.54533128, 46088390.160235666, 31680930.95472988],
 [72.52080313735902, 72.52383385963958, 72.52179466175681, 4.334019387293344],
 0.016812202907723228)

**2. 스케일링 MinMax**

In [52]:
from sklearn.preprocessing import MinMaxScaler

# 데이터에서 Y 제거
shop_X = shop_group.drop(['mean_money'],axis=1,inplace=False)

# StandardScaler 객체 생성
mm_scaler = MinMaxScaler()

# StandardScaler로 데이터 세트 변환
mm_scaler.fit(shop_X)
shop_mm= mm_scaler.transform(shop_X)

# 데이터프레임으로 변환
shop_df_mm = pd.DataFrame(data=shop_mm, columns=shop_X.columns)
shop_df_mm.describe()

,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액
count,63732.000000,63732.000000,63732.000000,63732.000000,63732.000000
mean,0.002234,0.002134,0.002734,0.002258,0.005546
std,0.009537,0.009688,0.010362,0.008547,0.016732
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000151,0.000145,0.000239,0.000139,0.000329
50%,0.000582,0.000563,0.000654,0.000571,0.001305
75%,0.001984,0.001901,0.002318,0.001967,0.004927
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
# 데이터 분리
y = shop_group['mean_money']
X = shop_df_mm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [54]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors(models)

LinearRegression RMSE : 112035333.002 / MAE : 46087972.919 / MAPE : 72.521 / R2 : 0.04
Ridge RMSE : 111920847.558 / MAE : 47321178.691 / MAPE : 75.742 / R2 : 0.042
Lasso RMSE : 112035621.442 / MAE : 46088445.306 / MAPE : 72.522 / R2 : 0.04
XGBRegressor RMSE : 113401830.924 / MAE : 31680953.217 / MAPE : 4.334 / R2 : 0.017


([112035333.00151774,
  111920847.55789173,
  112035621.44232838,
  113401830.92364556],
 [46087972.91853351, 47321178.69146059, 46088445.30624713, 31680953.21724263],
 [72.52054860584292, 75.74177572790029, 72.52200418462323, 4.334067440247355],
 0.016812153679671482)

**3. y값 정규분포화**

In [55]:
y_log = np.log1p(shop_group['mean_money'])

In [56]:
# 타겟값을 로그변환한 것을 사용했기 때문에 테스트 데이터 세트의 타겟값과 예측값을 로그변환 이전으로 돌려주는 식 추가
def get_rmse_log(model):
    pred = model.predict(X_test)
    y_test_exp = np.expm1(y_test)
    pred_exp = np.expm1(pred)
    
    mse = mean_squared_error(y_test_exp, pred_exp)
    rmse = np.sqrt(mse)
    return rmse
    
def get_mae_log(model):
    pred = model.predict(X_test)
    y_test_exp = np.expm1(y_test)
    pred_exp = np.expm1(pred)
    
    mae = mean_absolute_error(y_test_exp, pred_exp)
    return mae

def get_mape_log(model):
    pred = model.predict(X_test)
    y_test_exp = np.expm1(y_test)
    pred_exp = np.expm1(pred)
    
    mape = mean_absolute_percentage_error(y_test_exp, pred_exp)
    return mape

def get_r2_log(model):
    pred = model.predict(X_test)
    y_test_exp = np.expm1(y_test)
    pred_exp = np.expm1(pred)
    
    r2 = r2_score(y_test_exp, pred_exp)
    return r2

def check_errors_log(models):
    rmses = []
    maes = []
    mapes = []
    r2s = []
    
    for model in models:
        rmse = get_rmse_log(model)
        mae = get_mae_log(model)
        mape = get_mape_log(model)
        r2 = get_r2_log(model)
        
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        r2s.append(r2)
        
        print(model.__class__.__name__, '\n RMSE :', np.round(rmse, 3), '/ MAE :', np.round(mae, 3), '/ MAPE :', np.round(mape, 3), '/ R2 :', np.round(r2, 3))
    return rmses, maes, mapes, r2

In [57]:
# 데이터 분리
y = shop_group['mean_money']
X = shop_group.drop(['mean_money'],axis=1,inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=0)

In [58]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors_log(models)

LinearRegression 
 RMSE : 3.7569026432576646e+17 / MAE : 3334628893309276.0 / MAPE : 19114076.839 / R2 : -1.0790864771793439e+19
Ridge 
 RMSE : 3.75893871667061e+17 / MAE : 3336434938231050.0 / MAPE : 19124428.446 / R2 : -1.0802564274601865e+19
Lasso 
 RMSE : 3.758897791173181e+17 / MAE : 3336398628499945.5 / MAPE : 19124220.327 / R2 : -1.0802329049745551e+19
XGBRegressor 
 RMSE : 86636844.149 / MAE : 28356330.124 / MAPE : 0.743 / R2 : 0.426


([3.7569026432576646e+17,
  3.75893871667061e+17,
  3.758897791173181e+17,
  86636844.14889964],
 [3334628893309276.0,
  3336434938231050.0,
  3336398628499945.5,
  28356330.123626236],
 [19114076.83903057, 19124428.445548214, 19124220.32673951, 0.742703632212272],
 0.42614573096493047)

# 결과
y값을 정규분포화  
groupby('상권_코드', '서비스_업종_코드_명')  
XGBoost로 수행했을 때 가장 좋은 예측값 도출

## 원핫인코딩으로 수행

In [3]:
# 데이터 불러오기
shop_orig = pd.read_csv('shop_all.csv')
shop_all = shop_orig.copy()

# 점포수 0인것 1로 변경
shop_all['점포수'].replace(0, 1, inplace=True)
shop_all['점포수'].value_counts().sort_index()

# 필요한 컬럼 생성
shop_col = shop_all.copy()
shop_col['period'] = shop_col['기준_년_코드'].astype(str) + '-' + shop_col['기준_분기_코드'].astype(str)
shop_col['mean_money'] = shop_col['당월_매출_금액'] / shop_col['점포수']

# 필요없는 컬럼 삭제
shop_col.drop(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명',
           '서비스_업종_코드'], axis=1, inplace=True)

In [4]:
# 매출 금액만 남기고 모두 삭제
shop_drop3 = shop_col.copy()
shop_drop3.drop(['당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '월요일_매출_비율','화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
                '금요일_매출_비율', '토요일_매출_비율','일요일_매출_비율', '시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율',
                '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율', '연령대_10_매출_비율',
                '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율', '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_건수', 
                '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', 
                '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수', 
                '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', 
                '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '점포수', '월요일_매출_금액', '화요일_매출_금액', 
                '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', 
                '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', 
                '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'],
               axis=1, inplace=True)

shop_drop3.head()

,상권_코드,서비스_업종_코드_명,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,period,mean_money
0,1000428,제과점,284484215,198729159,85755056,106969928,169353641,2015-1,4.064060e+07
1,1000288,섬유제품,12005909,9736459,2269450,4839538,6868387,2015-1,3.001477e+06
2,1000270,자동차수리,71547516,50879960,20667556,44823579,17296999,2015-1,1.022107e+07
3,1000284,자동차수리,80195548,58480589,21714959,51194195,19312565,2015-1,1.002444e+07
4,1000287,편의점,1339105488,909912741,429192747,881901743,420784068,2015-1,1.217369e+08


In [5]:
# 분기를 인덱스 오름차순으로 설정
shop_drop3.index = shop_drop3['period']
shop_drop3 = shop_drop3.sort_index(ascending=True)
shop_drop3.drop(['period'], axis=1, inplace=True)
shop_drop3.head()

,상권_코드,서비스_업종_코드_명,당월_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,mean_money
period,,,,,,,,
2014-1,1000560,중식음식점,12363015,7190594,5172421,5191066,4598513,6.181508e+06
2014-1,1000725,제과점,316665247,234156208,82509039,118884530,168038070,3.958316e+07
2014-1,1000063,화초,52043466,36531072,15512394,16000724,26244832,6.505433e+06
2014-1,1000691,여행사,26048736,26048736,0,14443127,1977252,1.532279e+06
2014-1,1000070,화초,60814678,45302284,15512394,27877778,23138990,8.687811e+06


In [6]:
shop_drop3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1434810 entries, 2014-1 to 2020-2
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   상권_코드        1434810 non-null  int64  
 1   서비스_업종_코드_명  1434810 non-null  object 
 2   당월_매출_금액     1434810 non-null  int64  
 3   주중_매출_금액     1434810 non-null  int64  
 4   주말_매출_금액     1434810 non-null  int64  
 5   남성_매출_금액     1434810 non-null  int64  
 6   여성_매출_금액     1434810 non-null  int64  
 7   mean_money   1434810 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 98.5+ MB


In [1]:
from sklearn.preprocessing import OneHotEncoder

# 원핫인코딩 수행
# oh_enco = OneHotEncoder()
# shop_oh = oh_enco.fit_transform(shop_drop3)

shop_oh = pd.get_dummies(shop_drop3, columns=['서비스_업종_코드_명'], drop_first=True)

NameError: name 'pd' is not defined

In [8]:
print('원핫인코딩 수행 전 shape : ', shop_drop3.shape)
print('원핫인코딩 수행 후 shape : ', shop_oh.shape)

원핫인코딩 수행 전 shape :  (1434810, 8)
원핫인코딩 수행 후 shape :  (1434810, 89)


### 차원축소 Truncated SVD
1. 스케일링 적용 >> 스케일링 적용했더니 값이 너무 안나와서 적용안함
2. tsvd변환
3. tsvd변환 된 데이터 모델학습, 예측, 평가

In [9]:
# 차원축소 전 데이터 X, y로 나누기
shop_X = shop_oh.drop(['mean_money'],axis=1,inplace=False)
shop_y = shop_oh['mean_money']

In [37]:
from sklearn.decomposition import TruncatedSVD

# 원핫인코딩으로 인해 희소행렬이 되었기 때문에 PCA는 사용할 수 없고 SVD 사용
# 그중에서도 truncatedSVD 사용
tsvd = TruncatedSVD(n_components=4)

In [38]:
tsvd.fit(shop_X)
shop_tsvd = tsvd.transform(shop_X)
print('원핫인코딩 수행 후 shape : ', shop_oh.shape)
print('TruncatedSVD 수행 후 shap : ', shop_tsvd.shape)
print('컴포넌트 영향 : ', np.round(tsvd.explained_variance_ratio_, 4))

원핫인코딩 수행 후 shape :  (1434810, 89)
TruncatedSVD 수행 후 shap :  (1434810, 4)
컴포넌트 영향 :  [0.9573 0.0307 0.0089 0.0031]


In [41]:
# cumsum은 누적된 값 불러오는 함수
cumsum = np.cumsum(tsvd.explained_variance_ratio_)

# argmax는 해당 인덱스 불러옴 인덱스가 0부터 시작해서 +1 필요
d = np.argmax(cumsum >= 0.99999) + 1
print('선택할 차원 수 : ', d)

선택할 차원 수 :  4


### 모델 학습, 예측, 평가

In [39]:
# 데이터 분리
y = shop_y
X = shop_tsvd

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [40]:
# 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

rg_reg = Ridge()
rg_reg.fit(X_train, y_train)

ls_reg = Lasso()
ls_reg.fit(X_train, y_train)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

models = [lr_reg, rg_reg, ls_reg, xgb_reg]
check_errors(models)

LinearRegression RMSE : 114687396.028 / MAE : 46524744.536 / MAPE : 131.052 / R2 : 0.065
Ridge RMSE : 114687396.028 / MAE : 46524744.536 / MAPE : 131.052 / R2 : 0.065
Lasso RMSE : 114687396.028 / MAE : 46524744.536 / MAPE : 131.052 / R2 : 0.065
XGBRegressor RMSE : 88998677.683 / MAE : 31283274.343 / MAPE : 3.497 / R2 : 0.437


([114687396.02808376, 114687396.02808376, 114687396.02808376, 88998677.683244],
 [46524744.53582884, 46524744.53582882, 46524744.535828866, 31283274.34279958],
 [131.05173983580355,
  131.05173983580332,
  131.05173983580372,
  3.4970715393543577],
 0.43680352068674955)